In [1]:
#mengimport library yang diperlukan untuk membaca data
import pandas as pd
import numpy as np
df = pd.read_csv("SINGGALANG_.csv", sep = ';')

In [2]:
df.head(25)

,token_0,BIO_tag_0
0,Ia,O
1,menjabat,O
2,sebagai,O
3,Presiden,O
4,ketiga,O
5,Mesir,b-place
6,pada,O
7,periode,O
8,15,O
9,Oktober,O


In [3]:
# import library untuk preprocessing data
from nltk.tokenize import word_tokenize
import itertools
import re
import pandas as pd
import numpy as np
import string
from tqdm import tqdm

In [5]:
def convert_to_spaCyformat(df, listOfEntities):
    """
    Fungsi ini digunakan untuk mengubah BIO-tagged-DF kedalam format SpaCy Annotations
    """
    # memeriksa jika ditemukan value null
    assert not (df.iloc[:,0].isnull().any() or df.iloc[:,1].isnull().any())
    
    # membuat kamus kata kumpulan token kata dan tag
    dictTemp = {}
    dictTemp['token'] = np.array(df.iloc[:,0])
    dictTemp['BIO_tag'] = np.array(df.iloc[:,1].str.lower())
    dictTemp['indices'] = np.array([len(i) for i in dictTemp['token']])

    # pertama kita memerlukan index pertama token dan tag
    total_idx = [dictTemp['indices'][0]] 
    temp = dictTemp['indices'][0]
    
    # kemudian dilakukan perulangan untuk menghitung jumlah index
    for i in range(len(dictTemp['indices'])):
        if i > 0:
            temp += dictTemp['indices'][i]
            total_idx.append(temp)

    # membuat variabel dari start index
    dictTemp['start_idx'] = np.array([total_idx[i-1] if i > 0 else 0 for i in range(len(total_idx))])

    # membuat variabel dari last index
    dictTemp['end_idx'] = np.array(total_idx)
    del dictTemp['indices'] 

    enti = {}
    entities = []
    text = ''.join(dictTemp['token'])
    
    # mengombinasikan setiap daftar entity dengan awalan 'b-', 'i-', and 'e-' pada annotation
    listOfEntities = ['b-'+i.lower() for i in listOfEntities] + \
                     ['i-'+i.lower() for i in listOfEntities] + \
                     ['e-'+i.lower() for i in listOfEntities] + ['o']
    
    # memeriksa apakah ada tag pada entity
    error_tag = []
    error_boolean = []
    for i in np.unique(dictTemp['BIO_tag']):
        if i in listOfEntities:
            error_boolean.append(True)
        else:
            error_boolean.append(False)
            error_tag.append(i)
    assert all(error_boolean), "Some BIO-tag not listed in listOfEntities arg. {}".format(error_tag)
    
    # mengisi entitas dengan non 'O' annotations
    for row in range(len(dictTemp['token'])):
        if dictTemp['BIO_tag'][row] != 'o':
            entities.append((dictTemp['start_idx'][row], 
                             dictTemp['end_idx'][row], 
                             dictTemp['BIO_tag'][row]))

    
    start = []
    end = []
    BIO = []
    i = 0
    while i < len(entities):
        try:
            if entities[i][2][2:] == entities[i+1][2][2:]:
                if entities[i][2][0] is 'b':
#                     print('start1', entities[i][0])
                    start.append(entities[i][0])
                    i += 1
                    if entities[i][2][0] is 'e':
#                         print('end1a', entities[i][1])
                        end.append(entities[i][1])
                        BIO.append(entities[i][2][2:])
                        i += 1
                        continue
                    elif entities[i][2][0] is 'i':
                        for j in range(i, len(entities)):
                            if entities[j][2][0] is not 'e' and j < len(entities)-1:
#                                 print('sana', entities[j])
                                continue
                            elif entities[j][2][0] is 'e':
#                                 print('end1b', entities[j][1])
                                end.append(entities[j][1])
                                BIO.append(entities[j][2][2:])
                                i = j+1
                                break
                            else:
                                assert 1 == 0, \
                                    "Something error in the BIO-tag you wrote. Error BIO tag: '{}'" \
                                    .format(entities[j][2])
                    elif entities[i][2][0] is 'b':
#                         print('end1b', entities[i-1][1])
                        end.append(entities[i-1][1])
                        BIO.append(entities[i-1][2][2:])
                        continue
                        
#                         print('ss',i,j)
            else:
#                 print('start2a', entities[i][0], i)
                start.append(entities[i][0])
#                 print('end2a', entities[i][1], i)
                end.append(entities[i][1])
                BIO.append(entities[i][2][2:])
                i += 1
        except IndexError:
#             print('start2b', entities[i][0], i)
            start.append(entities[i][0])
#             print('end2b', entities[i][1], i)
            end.append(entities[i][1])
            BIO.append(entities[i][2][2:])
            i += 1

    enti['entities'] = [(i,j,k) for i,j,k in zip(start, end, BIO)]
    return [text, enti]

In [6]:
# annotations adalah setiap tag yang digunakan untuk memberi keterangan pada setiap entity
annotations = sorted(['organisation',
               'person',
               'place',
               ])
annotations

['organisation', 'person', 'place']

In [7]:
train_data = convert_to_spaCyformat(df, annotations)

### Training